# **Intro**

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 12.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
df1 = pd.read_csv('Fake_train (1).csv')
df2 = pd.read_csv('Fake_dev (1).csv')

In [ ]:
df1.head()

,text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake
1,Masha Allah,Fake
2,"അന്വേഷണം കഴിയുമ്പോൾ,. C. A. A. യ്ക്ക് എതിരായ ക...",Fake
3,Illathentha avaru purath vidayittalland verenth,Fake
4,Barana pakshathin matoru niyamam.nalla moyanth...,original


In [ ]:
df2.head()

,text,label
0,Full. Musilm. Verodamum,Fake
1,പക്ഷികളും മൃഗങ്ങളും ഈ ലോകത്ത് സുഖമായി ജീവിക്കു...,Fake
2,ഒരു താടിക്കാരൻ പാത്രം കൊട്ടാൻ പറഞ്ഞപ്പോ .........,original
3,കുംഭളേമ<br>മറന്നോ,original
4,ഇത് തിരുവാതിര അല്ല...... കോറോണയെ കൈകൊട്ടി കൊല്...,original


In [ ]:
df1.isnull().sum()

,0
text,0
label,0


In [ ]:
df2.isnull().sum()

,0
text,0
label,0


In [ ]:
import re
def preprocessing(df):
    url_pattern = r'https?://\S+|www\.\S+'

    def count_urls(text):
        return len(re.findall(url_pattern, text))

    df['hashtag_count'] = df['text'].apply(lambda x: x.count('#'))
    df['mentions_count'] = df['text'].apply(lambda x: x.count('@'))
    df['dollar_count'] = df['text'].apply(lambda x: x.count('$'))
    df['url_count'] = df['text'].apply(count_urls)
    return df

df1 = preprocessing(df1)
df2 = preprocessing(df2)

In [ ]:
print(df1['hashtag_count'].sum())
print(df1['mentions_count'].sum())
print(df1['url_count'].sum())

105
2
29


In [ ]:
df1.head()

,text,label,hashtag_count,mentions_count,dollar_count,url_count
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake,0,0,0,0
1,Masha Allah,Fake,0,0,0,0
2,"അന്വേഷണം കഴിയുമ്പോൾ,. C. A. A. യ്ക്ക് എതിരായ ക...",Fake,0,0,0,0
3,Illathentha avaru purath vidayittalland verenth,Fake,0,0,0,0
4,Barana pakshathin matoru niyamam.nalla moyanth...,original,0,0,0,0


In [ ]:
print(df2['hashtag_count'].sum())
print(df2['mentions_count'].sum())
print(df2['url_count'].sum())

43
0
3


In [ ]:
df2.head()

,text,label,hashtag_count,mentions_count,dollar_count,url_count
0,Full. Musilm. Verodamum,Fake,0,0,0,0
1,പക്ഷികളും മൃഗങ്ങളും ഈ ലോകത്ത് സുഖമായി ജീവിക്കു...,Fake,0,0,0,0
2,ഒരു താടിക്കാരൻ പാത്രം കൊട്ടാൻ പറഞ്ഞപ്പോ .........,original,0,0,0,0
3,കുംഭളേമ<br>മറന്നോ,original,0,0,0,0
4,ഇത് തിരുവാതിര അല്ല...... കോറോണയെ കൈകൊട്ടി കൊല്...,original,0,0,0,0


In [ ]:
import re
import emoji

def remove_urls(text):
    """Remove URLs from the given text."""
    url_pattern = r'https?://\S+|www\.\S+'
    return re.sub(url_pattern, '', text)
def handle_emojis(text):
    # Remove emojis
    text = emoji.replace_emoji(text, '')
    # OR keep emojis as text
    # text = emoji.demojize(text)
    return text
def remove_hashtags(text):
    """Remove hashtags from the given text."""
    if '#' in text:
        text = text.replace('#', '')
    hashtag_pattern = r'#\w+'
    return re.sub(hashtag_pattern, '', text)


def remove_mentions(text):
    """Remove mentions from the given text."""
    # Remove mentions with no space before
    if '@' in text:
        text = text.replace('@', '')

    mention_pattern = r'@\S+\b'
    text = re.sub(mention_pattern, '', text)
    # Check if the mention is at the end of the text
    if text.endswith(' @'):
        text = text[:-2]  # Remove the space and '@'

    return text
def remove_dollars(text):
    """Remove $ from the given text."""
    if "$" in text:
        text = text.replace('$','')
    return text
def remove_consecutive_punctuation(text):
    """Remove consecutive punctuation from the given text."""
    text = re.sub(r'([.,!?;:|]){2,}', r'\1', text)
    return str(text).lower()

def processing(text):
    """Clean the given tweet by removing URLs, hashtags, and mentions."""
    cleaned_text = remove_urls(text)
    cleaned_text = handle_emojis(cleaned_text)
    cleaned_text = remove_hashtags(cleaned_text)
    cleaned_text = remove_mentions(cleaned_text)
    cleaned_text = remove_dollars(cleaned_text)
    cleaned_text = remove_consecutive_punctuation(cleaned_text)
    return cleaned_text

In [ ]:
df1['text'] = df1['text'].apply(lambda x: processing(x))
df2['text'] = df2['text'].apply(lambda x: processing(x))

In [ ]:
df1['Length'] = df1['text'].apply(lambda x: len(x.split(" ")))
df2['Length'] = df2['text'].apply(lambda x: len(x.split(" ")))

In [ ]:
df1.head()

,text,label,hashtag_count,mentions_count,dollar_count,url_count,Length
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake,0,0,0,0,5
1,masha allah,Fake,0,0,0,0,2
2,അന്വേഷണം കഴിയുമ്പോൾ. c. a. a. യ്ക്ക് എതിരായ കല...,Fake,0,0,0,0,16
3,illathentha avaru purath vidayittalland verenth,Fake,0,0,0,0,5
4,barana pakshathin matoru niyamam.nalla moyanth...,original,0,0,0,0,6


In [ ]:
df1.drop(columns=['hashtag_count', 'mentions_count', 'url_count', 'dollar_count','Length'], inplace=True)
df2.drop(columns=['hashtag_count', 'mentions_count', 'url_count', 'dollar_count','Length'], inplace=True)

In [ ]:
df1.head()

,text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake
1,masha allah,Fake
2,അന്വേഷണം കഴിയുമ്പോൾ. c. a. a. യ്ക്ക് എതിരായ കല...,Fake
3,illathentha avaru purath vidayittalland verenth,Fake
4,barana pakshathin matoru niyamam.nalla moyanth...,original


In [ ]:
# # data preprocessing
# # tokenization
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
# df1['text'] = df1['text'].apply(lambda x: tokenizer.tokenize(x))
# df2['text'] = df2['text'].apply(lambda x: tokenizer.tokenize(x))

In [ ]:
!pip install indicnlp

In [ ]:
#label encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df1['label'] = label_encoder.fit_transform(df1['label'])
df2['label'] = label_encoder.fit_transform(df2['label'])


In [ ]:
df1.head()

,text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,0
1,masha allah,0
2,അന്വേഷണം കഴിയുമ്പോൾ. c. a. a. യ്ക്ക് എതിരായ കല...,0
3,illathentha avaru purath vidayittalland verenth,0
4,barana pakshathin matoru niyamam.nalla moyanth...,1


In [ ]:
#spliting dataset
from sklearn.model_selection import train_test_split
X = df1['text']  # Features
y = df1['label']  # Labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#Feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)  # Limit to top 5000 words
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# **ML models**

In [ ]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1-score:", f1_score(y_test, y_pred, average='weighted'))


Accuracy: 0.7730061349693251
Precision: 0.7785130970277031
Recall: 0.7730061349693251
F1-score: 0.7728352380259556


In [ ]:
#SVM
from sklearn.svm import SVC

model = SVC(kernel='linear')
model.fit(X_train_tfidf, y_train)

# Predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1-score:", f1_score(y_test, y_pred, average='weighted'))


In [ ]:
# RandomForest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

# Predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1-score:", f1_score(y_test, y_pred, average='weighted'))


Accuracy: 0.7331288343558282
Precision: 0.7486040846381589
Recall: 0.7331288343558282
F1-score: 0.7311695685580721


In [ ]:
#xgboost
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Model creation
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)

# Train the model
model.fit(X_train_tfidf, y_train)

# Predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1-score:", f1_score(y_test, y_pred, average='weighted'))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:55:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.7576687116564417
Precision: 0.7733262668765191
Recall: 0.7576687116564417
F1-score: 0.7560207600186073
